# Задание

В этом задании вам предстоит продолжить работу с датасетом lenta-ru-news для той же задачи - классификации текстов по топикам. Можно переиспользовать подготовленные данные из ДЗ 1 или загрузить их заново.

1. Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`
2. Обучите word2vec-эмбеддинги с помощью библиотеки gensim - **2 балла**
    - создайте модель для обучения на ваших данных, опишите, какими значениями вы инициализировали гиперпараметры модели, и почему
    - визуально оцените внутреннее (intrinsic) качество получившихся эмбеддингов, используя методы gensim - doesnt_match, most_similar
3. Загрузите предобученные эмбеддинги из navec и rusvectores (на ваш вкус) - **1 балл**
4. Обучите модель `sklearn.linear_model.LogisticRegression` с тремя вариантами векторизации текстов и сравните их качество между собой на валидационной выборке: **2 балла**
    - ваши эмбеддинги w2v
    - предобученные эмбеддинги navec
    - предобученные эмбеддинги rusvectores
5. Попробуйте улучшить качество модели, взяв для ее обучения лучший набор эмбеддингов и используя его с взвешиванием через tf-idf. То есть, необходимо каждый текст представить в виде взвешенного усреднения эмбеддингов его слов, где весами являются соответствующие коэффициенты tf-idf - **2 балла**
6. Финально сравните качество всех моделей на тестовой выборке - **1 балл**


**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

## Этап 0 - Подготовка

### Импортируем необходимые библиотеки и компоненты

In [77]:
import random
import warnings
from typing import Any

import polars as pl
import numpy as np
import numpy.typing as npt
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec, KeyedVectors
from navec import Navec
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger

warnings.filterwarnings("ignore")

### Фиксируем seed'ы

In [2]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# Этап 1 - Загрузка и разделение датасета

In [3]:
df = pl.read_parquet("../data/processed/hw_1.parquet")

print(df.shape)
df.sample(3)

(100000, 4)


topic,content,clean_content,encoded_topic
str,str,str,i64
"""Россия""","""ООН попросила Россию отложить …","""оон попросить россия отложить …",10
"""Силовые структуры""","""Присяжные удалились для вынесе…","""присяжный удалиться вынесение …",11
"""Спорт""","""Футболист ЦСКА посоветовал игр…","""футболист цска посоветовать иг…",12


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df["encoded_topic"], random_state=RANDOM_STATE)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["encoded_topic"], random_state=RANDOM_STATE)

print(f"Train size: {train_df.shape}, val size: {val_df.shape}, test size {test_df.shape}")

Train size: (60000, 4), val size: (20000, 4), test size (20000, 4)


### Токенизируем тексты для W2V

In [5]:
train_df = train_df.with_columns(pl.col("clean_content").str.split(" ").alias("tokens"))

In [6]:
train_df.sample(3)

topic,content,clean_content,encoded_topic,tokens
str,str,str,i64,list[str]
"""Бывший СССР""","""В ДНР продемонстрировали «отве…","""днр продемонстрировать ответ а…",2,"[""днр"", ""продемонстрировать"", … ""беспилотник""]"
"""Мир""","""Журналист The Sun пронес муляж…","""журналист пронести муляж бомба…",7,"[""журналист"", ""пронести"", … ""добавлять""]"
"""Россия""","""Прокуратура начала проверку в …","""прокуратура начать проверка си…",10,"[""прокуратура"", ""начать"", … ""налог""]"


## Этап 2 - Обучение W2V модели

Возьмём стандартные параметры для обучения, но сделаем чуть меньше эпох для ускорения процесса обучения.

In [8]:
w2v_model = Word2Vec(
    sentences=train_df["tokens"].to_list(),
    vector_size=300,
    window=5,
    min_count=5,
    sg=1,
    epochs=5,
    negative=5,
    seed=RANDOM_STATE,
    workers=4,
)

In [9]:
w2v_model.save("../models/w2v_1203.model")

In [13]:
print("Слова, похожие на 'март':")
print(w2v_model.wv.most_similar("март", topn=5))

Слова, похожие на 'март':
[('апрель', 0.8195676803588867), ('февраль', 0.7926869988441467), ('май', 0.7555468082427979), ('декабрь', 0.7477976083755493), ('январь', 0.729356050491333)]


In [15]:
print("Слово, не подходящее в группе:")
print(w2v_model.wv.doesnt_match(["март", "декабрь", "январь", "игра"]))

Слово, не подходящее в группе:
игра


## Этап 3 - Загрузим предобученные эмбеддинги

In [19]:
navec_model = Navec.load("../models/navec/navec_hudlit_v1_12B_500K_300d_100q.tar")

In [22]:
navec_model.sim("март", "апрель")

0.8772458

In [32]:
rusvectores_model = KeyedVectors.load_word2vec_format("../models/rusvectors/model.bin", binary=True)

In [37]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)


def transform_word(word: str) -> str:
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    if doc.tokens:
        token = doc.tokens[0]
        if token.pos:
            return f"{token.text}_{token.pos}"

    return word

In [40]:
rusvectores_model.most_similar(transform_word("март"), topn=5)

[('апрель_NOUN', 0.9572353959083557),
 ('февраль_NOUN', 0.9567031264305115),
 ('ноябрь_NOUN', 0.9381636381149292),
 ('октябрь_NOUN', 0.9329459071159363),
 ('май_NOUN', 0.9289151430130005)]

## Этап 4 - Обучение моделей

Напишем методы для получения эмбеддинга предложения - берём среднее по эмбеддингам слов, наиболее популярный подход.

In [53]:
def document_embedding_mean(tokens: list[str], emb_model: Any, apply=lambda x: x) -> npt.NDArray[np.float32]:
    vecs = [emb_model[apply(word)] for word in tokens if apply(word) in emb_model]
    if not vecs:
        return np.zeros(emb_model.vector_size)

    return np.mean(vecs, axis=0)

In [71]:
def get_embeddings(df: pl.DataFrame, emb_model: Any, column_name: str, apply=lambda x: x) -> npt.NDArray[np.float32]:
    embeddings = []
    if column_name not in df.columns:
        df = df.with_columns(pl.col("clean_content").str.split(" ").alias("tokens"))

    for tokens in tqdm(df[column_name].to_list()):
        embeddings.append(document_embedding_mean(tokens, emb_model, apply))

    return np.array(embeddings)

In [64]:
X_train_w2v = get_embeddings(train_df, w2v_model.wv, "tokens")
X_val_w2v = get_embeddings(val_df, w2v_model.wv, "tokens")
X_test_w2v = get_embeddings(test_df, w2v_model.wv, "tokens")

In [65]:
X_train_navec = get_embeddings(train_df, navec_model, "tokens")
X_val_navec = get_embeddings(val_df, navec_model, "tokens")
X_test_navec = get_embeddings(test_df, navec_model, "tokens")

In [72]:
X_train_rusvect = get_embeddings(train_df, rusvectores_model, "tokens", transform_word)
X_val_rusvect = get_embeddings(val_df, rusvectores_model, "tokens", transform_word)
X_test_rusvect = get_embeddings(test_df, rusvectores_model, "tokens", transform_word)

  0%|          | 0/60000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [73]:
np.save("X_train_rusvect.npy", X_train_rusvect)
np.save("X_val_rusvect.npy", X_val_rusvect)
np.save("X_test_rusvect.npy", X_test_rusvect)

In [74]:
y_train = train_df["encoded_topic"].to_list()
y_val = val_df["encoded_topic"].to_list()
y_test = test_df["encoded_topic"].to_list()

### Методы для обучения модели

Используем cv=3 и параметры из hw_1

In [75]:
def train_evaluate_cv(X, y, cv=3):
    clf = LogisticRegression(solver="lbfgs", C=10, max_iter=1000, random_state=RANDOM_STATE)
    scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
    print("CV scores:", scores)
    print("Average CV accuracy:", scores.mean())
    clf.fit(X, y)
    return scores.mean(), clf

In [ ]:
acc_w2v, clf_w2v = train_evaluate_cv(X_train_w2v, y_train)
print(f"Accuracy (свой w2v): {acc_w2v:.3f}")
val_result_w2v = clf_w2v.predict(X_val_w2v)
print(f"Accuracy (свой w2v) валидация: {accuracy_score(y_val, val_result_w2v):.3f}")

CV scores: [0.7939 0.7943 0.7911]
Average CV accuracy: 0.7931
Accuracy (свой w2v): 0.793
Accuracy (свой w2v): 0.795


In [80]:
acc_navec, clf_navec = train_evaluate_cv(X_train_navec, y_train)
print(f"Accuracy (navec): {acc_navec:.3f}")
val_result_navec = clf_navec.predict(X_val_navec)
print(f"Accuracy (navec) валидация: {accuracy_score(y_val, val_result_navec):.3f}")

CV scores: [0.7645  0.76365 0.7627 ]
Average CV accuracy: 0.7636166666666667
Accuracy (navec): 0.764
Accuracy (navec) валидация: 0.771


In [81]:
acc_rusvect, clf_rusvect = train_evaluate_cv(X_train_rusvect, y_train)
print(f"Accuracy (rusvectores): {acc_rusvect:.3f}")
val_result_rusvect = clf_rusvect.predict(X_val_rusvect)
print(f"Accuracy (navec) валидация: {accuracy_score(y_val, val_result_rusvect):.3f}")

CV scores: [0.74305 0.7376  0.7383 ]
Average CV accuracy: 0.73965
Accuracy (rusvectores): 0.740
Accuracy (navec) валидация: 0.748


### Выводы

Несколько удивительно, но наша модель W2V показала себя лучше, чем предобученные на большом корпусе варианты.

Возможно, отобранные новости имеют специфический домен или наша предобработка нерелевантна для других векторов.

## Этап 5 - Улучшение результатов с помощью TFIDF

In [82]:
tfidf = TfidfVectorizer()
tfidf.fit(train_df["clean_content"])

TfidfVectorizer()

In [87]:
def weighted_document_embedding(
    text: str,
    emb_model: Any,
    tfidf_vectorizer: TfidfVectorizer,
) -> npt.NDArray[np.float32]:
    tokens = text.split()
    tfidf_vec = tfidf_vectorizer.transform([" ".join(tokens)]).toarray()[0]

    word2tfidf = {}

    for token in tokens:
        if token in tfidf_vectorizer.vocabulary_:
            idx = tfidf_vectorizer.vocabulary_[token]
            word2tfidf[token] = tfidf_vec[idx]

    vecs, weights = [], []
    model_vocab = emb_model.wv if hasattr(emb_model, "wv") else emb_model
    for token in tokens:
        if token in model_vocab and token in word2tfidf:
            vecs.append(model_vocab[token] * word2tfidf[token])
            weights.append(word2tfidf[token])

    if not vecs:
        return np.zeros(emb_model.vector_size)

    return np.sum(vecs, axis=0) / np.sum(weights)


def get_weighted_embeddings(
    df: pl.DataFrame,
    emb_model: Any,
    tfidf_vectorizer: TfidfVectorizer,
) -> npt.NDArray[np.float32]:
    embeddings = []
    for text in df["clean_content"]:
        embeddings.append(weighted_document_embedding(text, emb_model, tfidf_vectorizer))

    return np.array(embeddings)

In [88]:
X_train_w2v_tfidf = get_weighted_embeddings(train_df, w2v_model, tfidf)
X_val_w2v_tfidf = get_weighted_embeddings(val_df, w2v_model, tfidf)
X_test_w2v_tfidf = get_weighted_embeddings(test_df, w2v_model, tfidf)

In [90]:
acc_w2v_tfidf, clf_w2v_tfidf = train_evaluate_cv(X_train_w2v_tfidf, y_train)

CV scores: [0.767  0.7692 0.7665]
Average CV accuracy: 0.7675666666666666


In [91]:
val_result_w2v_tfidf = clf_w2v_tfidf.predict(X_val_w2v_tfidf)
print(f"Accuracy (свой w2v + TFIDF): {accuracy_score(y_val, val_result_w2v_tfidf):.3f}")

Accuracy (свой w2v + TFIDF): 0.766


### Вывод

К сожалению, подход с TFIDF не улучшил качество эмбеддингов предложений. 

Можно сделать вывод, что даже простая эвристика в виде mean бывает весьма эффективна.

### Этап 6 - Финальная оценка на тестовой выборке

In [92]:
def evaluate_on_test(clf: Any, X_test: npt.NDArray[np.float32], y_test: npt.NDArray[np.float32]) -> float:
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [93]:
test_acc_w2v = evaluate_on_test(clf_w2v, X_test_w2v, y_test)
test_acc_navec = evaluate_on_test(clf_navec, X_test_navec, y_test)
test_acc_rusvect = evaluate_on_test(clf_rusvect, X_test_rusvect, y_test)
test_acc_navec_tfidf = evaluate_on_test(clf_w2v_tfidf, X_test_w2v_tfidf, y_test)

In [94]:
print(f"Test Accuracy (свой w2v): {test_acc_w2v:.3f}")
print(f"Test Accuracy (navec): {test_acc_navec:.3f}")
print(f"Test Accuracy (rusvectores): {test_acc_rusvect:.3f}")
print(f"Test Accuracy (navec + TFIDF): {test_acc_navec_tfidf:.3f}")

Test Accuracy (свой w2v): 0.797
Test Accuracy (navec): 0.769
Test Accuracy (rusvectores): 0.746
Test Accuracy (navec + TFIDF): 0.766


### Вывод

На отложенной тестовой выборке победителем по качеству также является наш обученный W2V без TFIDF.